In [1]:
import os 
import pickle
import pandas as pd

In [2]:
MAIN_PATH = '../../dumps'

In [ ]:
BoostLGBM

In [23]:
def load_datasets(df_name, algo_name, append_test=True):
    with open(os.path.join(MAIN_PATH, df_name, f'datalog_lama_fit_predict_{algo_name}.pickle'), "rb") as f:
        lama_data = pickle.load(f)
    
    with open(os.path.join(MAIN_PATH, df_name, f'datalog_spark_fit_predict_{algo_name}.pickle'), "rb") as f:
        spark_data = pickle.load(f)
    
    lama_df = lama_data['data']['train'].to_pandas().data
    spark_df = spark_data['data']['train'].to_pandas().data
    print(len(list(lama_df)))
    print(len(list(spark_df)))
    print(list(lama_df))
    print(list(spark_df))
    print()
    print('lama and Spark', set(lama_df).intersection(set(spark_df)))
    print()
    print('lama - Spark', set(lama_df).difference(set(spark_df)))
    print()
    print('Spark - lama', set(spark_df).difference(set(lama_df)))
    
    if append_test:
        with open(os.path.join(MAIN_PATH, df_name,f'datalog_lama_predict_{algo_name}.pickle'), 'rb') as f:
            lama_test_data = pickle.load(f)
        with open(os.path.join(MAIN_PATH, df_name,f'datalog_spark_predict_{algo_name}.pickle'), 'rb') as f:
            spark_test_data = pickle.load(f)
        
        lama_test_df = lama_test_data['data']['predict'].to_pandas().data
        spark_test_df = spark_test_data['data']['predict'].to_pandas().data
        
        lama_df = pd.concat([lama_df, lama_test_df])
        spark_df = pd.concat([spark_df, spark_test_df])
        
        assert lama_df.shape[0] == spark_df.shape[0]
    return lama_df, spark_df

In [17]:
stats_names = ['count', 'mean', 
               'std', 'min', 
               '25%', '50%',
               '75%', 'max']

def compare_datasets(lama_df, spark_df, diff_proc=0.05):
    lama_df_stats = lama_df.describe()
    spark_df_stats = spark_df.describe()
    columns = list(lama_df_stats)
    for col in columns:
        if col not in list(spark_df):
            print(col)
            spark_df = spark_df.rename(columns={'oof__'+col:col})
            spark_df_stats = spark_df.describe()

        lama_col_uniques = lama_df[col].unique()
        spark_col_uniques = spark_df[col].unique()
        lama_col_uniques_num = len(lama_col_uniques)
        spark_col_uniques_num = len(spark_col_uniques)
        # comparing uniques:
        if abs(lama_col_uniques_num - spark_col_uniques_num) > lama_col_uniques_num * diff_proc:
            print()
            print(f'Difference between uniques {lama_col_uniques_num} (lama) and {spark_col_uniques_num} (spark)')
#             print('Lama: ', lama_col_uniques)
#             print('Spark: ', spark_col_uniques)
            print()
        for stats_col in stats_names:
            if abs(lama_df_stats[col][stats_col] - spark_df_stats[col][stats_col]) > lama_df_stats[col][stats_col] * diff_proc:
                print(f'Difference in col {col} and stats {stats_col} between {lama_df_stats[col][stats_col]} (lama) and {spark_df_stats[col][stats_col]} (spark)')
                

In [21]:
with open('../../dumps/datalogs_used_cars_dataset_42/datalog_lama_reader_fit_read.pickle', "rb") as f:
    lama_data = pickle.load(f)
    
with open('../../dumps/datalogs_used_cars_dataset_42/datalog_spark_reader_fit_read.pickle', "rb") as f:
    spark_data = pickle.load(f)

In [22]:
lama_cols = list(lama_data['data']['train'].to_pandas().data)
spark_cols = list(spark_data['data']['train'].to_pandas().data)

assert len(lama_cols) == len(spark_cols)

# for i in missing_columns:
#     if i in spark_cols and i in lama_cols:
#         continue
#     print(f'col {i} in spark ({i in spark_cols}) and in lama ({i in lama_cols})')

# Buzz dataset (lgbm)

In [13]:
lama_df, spark_df = load_datasets('datalogs_buzz_dataset_42', 'BoostLGBM')

FileNotFoundError: [Errno 2] No such file or directory: '../../dumps/datalogs_buzz_dataset_42/datalog_lama_fit_predict_BoostLGBM.pickle'

In [62]:
compare_datasets(lama_df, spark_df)

Difference in col ADL_0 and stats std between 1.1567665338516235 (lama) and 1.3999865055084229 (spark)
Difference in col ADL_0 and stats max between 62.0 (lama) and 136.0 (spark)
Difference in col ADL_2 and stats std between 1.0269153118133545 (lama) and 0.8860918283462524 (spark)
Difference in col ADL_4 and stats std between 0.9910610318183899 (lama) and 0.9011517763137817 (spark)
Difference in col ADL_5 and stats std between 0.926908016204834 (lama) and 0.9959810972213745 (spark)
Difference in col ADL_5 and stats max between 42.0 (lama) and 58.0 (spark)
Difference in col AI_0 and stats std between 392.9698791503906 (lama) and 371.04229736328125 (spark)
Difference in col AI_1 and stats std between 425.3762512207031 (lama) and 401.0068664550781 (spark)
Difference in col AI_6 and stats std between 829.3038940429688 (lama) and 702.764892578125 (spark)
Difference in col ASNAC_2 and stats max between 0.019068999215960503 (lama) and 0.021774999797344208 (spark)
Difference in col ASNAC_6 and

# used cars dataset (lgbm)

In [18]:
lama_df, spark_df = load_datasets('datalogs_used_cars_dataset_42', 'BoostLGBM')

FileNotFoundError: [Errno 2] No such file or directory: '../../dumps/datalogs_used_cars_dataset_42/datalog_lama_fit_predict_BoostLGBM.pickle'

In [180]:
lama_df, spark_df = load_datasets('datalogs_used_cars_dataset_42', 'datalog_lama_fit_predict_BoostLGBM.pickle','BoostLGBM', 'datalog_spark_fit_predict_BoostLGBM.pickle', False)

59
59
['oof__le__back_legroom', 'oof__le__bed_length', 'oof__le__city', 'oof__le__engine_cylinders', 'oof__le__engine_type', 'oof__le__franchise_make', 'oof__le__front_legroom', 'oof__le__fuel_tank_volume', 'oof__le__height', 'oof__le__length', 'oof__le__listing_color', 'oof__le__make_name', 'oof__le__maximum_seating', 'oof__le__model_name', 'oof__le__power', 'oof__le__torque', 'oof__le__transmission_display', 'oof__le__wheelbase', 'oof__le__width', 'oof__inter__(power__torque)', 'oof__inter__(power__model_name)', 'oof__inter__(power__length)', 'oof__inter__(torque__model_name)', 'oof__inter__(torque__length)', 'oof__inter__(model_name__length)', 'oof__inter__(power__torque__model_name)', 'oof__inter__(power__torque__length)', 'oof__inter__(power__model_name__length)', 'oof__inter__(torque__model_name__length)', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'highway_fuel_economy', 'horsepower', 'latitude', 'listing_id', 'longitude', 'mileage', 'owner_count', 'savings_amou

In [181]:
compare_datasets(lama_df, spark_df)

Difference in col oof__le__back_legroom and stats std between 8305.716796875 (lama) and 7678.8427734375 (spark)
Difference in col oof__le__back_legroom and stats min between 6965.07666015625 (lama) and 7446.353515625 (spark)
Difference in col oof__le__back_legroom and stats max between 142416.71875 (lama) and 75284.1015625 (spark)
Difference in col oof__le__bed_length and stats min between 11476.6416015625 (lama) and 10885.9970703125 (spark)

Difference between uniques 57 (lama) and 60 (spark)

Difference in col oof__le__engine_cylinders and stats max between 56099.32421875 (lama) and 64803.98828125 (spark)

Difference between uniques 57 (lama) and 60 (spark)

Difference in col oof__le__engine_type and stats max between 56099.32421875 (lama) and 64803.98828125 (spark)
Difference in col oof__le__fuel_tank_volume and stats std between 8400.8681640625 (lama) and 8860.0400390625 (spark)
Difference in col oof__le__fuel_tank_volume and stats min between 7687.4736328125 (lama) and 5591.065917

In [130]:
missing_columns = set(lama_df).difference(set(spark_df))

# used cars dataset (l2)

In [24]:
lama_df, spark_df = load_datasets('datalogs_used_cars_dataset_42', 'LinearLBFGS')

66
66
['scaler__fillnamed__fillinf__oof__le__back_legroom', 'scaler__fillnamed__fillinf__oof__le__city', 'scaler__fillnamed__fillinf__oof__le__front_legroom', 'scaler__fillnamed__fillinf__oof__le__fuel_tank_volume', 'scaler__fillnamed__fillinf__oof__le__height', 'scaler__fillnamed__fillinf__oof__le__length', 'scaler__fillnamed__fillinf__oof__le__model_name', 'scaler__fillnamed__fillinf__oof__le__power', 'scaler__fillnamed__fillinf__oof__le__torque', 'scaler__fillnamed__fillinf__oof__le__vin', 'scaler__fillnamed__fillinf__oof__le__wheelbase', 'scaler__fillnamed__fillinf__oof__le__width', 'scaler__fillnamed__fillinf__oof__inter__(bed_height__franchise_dealer)', 'scaler__fillnamed__fillinf__oof__inter__(bed_height__is_cpo)', 'scaler__fillnamed__fillinf__oof__inter__(bed_height__is_new)', 'scaler__fillnamed__fillinf__oof__inter__(franchise_dealer__is_cpo)', 'scaler__fillnamed__fillinf__oof__inter__(franchise_dealer__is_new)', 'scaler__fillnamed__fillinf__oof__inter__(is_cpo__is_new)', 'sca

In [25]:
compare_datasets(lama_df, spark_df)

Difference in col scaler__fillnamed__fillinf__oof__le__back_legroom and stats mean between -0.01236013974994421 (lama) and -0.0034597343765199184 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__back_legroom and stats min between -2.503246545791626 (lama) and -2.6465444564819336 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__back_legroom and stats 25% between -0.6342592835426331 (lama) and -0.6532771587371826 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__back_legroom and stats 50% between -0.2571200728416443 (lama) and -0.26976844668388367 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__back_legroom and stats max between 14.086370468139648 (lama) and 6.4137163162231445 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__city and stats mean between -0.011443083174526691 (lama) and -0.0004485786485020071 (spark)
Difference in col scaler__fillnamed__fillinf__oof__le__city and stats min between -2.234984874725342 (lam

KeyError: 'scaler__fillnamed__fillinf__oof__inter__(bed_height__franchise_dealer)'

# Lama test df (lgbm)

In [182]:
lama_df, spark_df = load_datasets('datalogs_lama_test_dataset_42', 'BoostLGBM', True)

110
115
['oof__le__OCCUPATION_TYPE', 'oof__le__ORGANIZATION_TYPE', 'oof__inter__(ORGANIZATION_TYPE__OCCUPATION_TYPE)', 'oof__inter__(ORGANIZATION_TYPE__NAME_TYPE_SUITE)', 'oof__inter__(ORGANIZATION_TYPE__WALLSMATERIAL_MODE)', 'oof__inter__(OCCUPATION_TYPE__NAME_TYPE_SUITE)', 'oof__inter__(OCCUPATION_TYPE__WALLSMATERIAL_MODE)', 'oof__inter__(NAME_TYPE_SUITE__WALLSMATERIAL_MODE)', 'oof__inter__(ORGANIZATION_TYPE__OCCUPATION_TYPE__NAME_TYPE_SUITE)', 'oof__inter__(ORGANIZATION_TYPE__OCCUPATION_TYPE__WALLSMATERIAL_MODE)', 'oof__inter__(ORGANIZATION_TYPE__NAME_TYPE_SUITE__WALLSMATERIAL_MODE)', 'oof__inter__(OCCUPATION_TYPE__NAME_TYPE_SUITE__WALLSMATERIAL_MODE)', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'APARTMENTS_AVG', 'APARTMENTS_MEDI', 'APARTMENTS_MODE', 'BASEMENTAREA_AVG', 'BASEMENTAREA_

In [183]:
compare_datasets(lama_df, spark_df)

Difference in col oof__le__OCCUPATION_TYPE and stats mean between 0.0809224396944046 (lama) and 0.9182155132293701 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats std between 0.016261043027043343 (lama) and 0.01464250311255455 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats min between 0.05422908812761307 (lama) and 0.8962379097938538 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats 25% between 0.0672389343380928 (lama) and 0.9030088782310486 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats 50% between 0.07304907590150833 (lama) and 0.9259472489356995 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats 75% between 0.09828771650791168 (lama) and 0.9305777549743652 (spark)
Difference in col oof__le__OCCUPATION_TYPE and stats max between 0.1142038032412529 (lama) and 0.94310462474823 (spark)
Difference in col oof__le__ORGANIZATION_TYPE and stats mean between 0.08125874400138855 (lama) and 0.9180808663368225 (spark)
Difference in c

# Lama test df (l2)

Lama:
```
OOf on train metric
['0.73']
Test metric
['0.75']
```
Spark:
```
OOf on train metric
['0.49']
Test metric
['0.48']
```

In [27]:
lama_df, spark_df = load_datasets('datalogs_lama_test_dataset_42', 'LinearLBFGS', True)

172
172
['scaler__fillnamed__fillinf__AMT_ANNUITY', 'scaler__fillnamed__fillinf__AMT_CREDIT', 'scaler__fillnamed__fillinf__AMT_GOODS_PRICE', 'scaler__fillnamed__fillinf__AMT_INCOME_TOTAL', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_DAY', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_HOUR', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_MON', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_QRT', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_WEEK', 'scaler__fillnamed__fillinf__AMT_REQ_CREDIT_BUREAU_YEAR', 'scaler__fillnamed__fillinf__APARTMENTS_AVG', 'scaler__fillnamed__fillinf__APARTMENTS_MEDI', 'scaler__fillnamed__fillinf__APARTMENTS_MODE', 'scaler__fillnamed__fillinf__BASEMENTAREA_AVG', 'scaler__fillnamed__fillinf__BASEMENTAREA_MEDI', 'scaler__fillnamed__fillinf__BASEMENTAREA_MODE', 'scaler__fillnamed__fillinf__CNT_CHILDREN', 'scaler__fillnamed__fillinf__CNT_FAM_MEMBERS', 'scaler__fillnamed__fillinf__COMMONAREA_AVG', 'scaler__fillnamed__fillinf__COMMONAREA_MED

In [28]:
compare_datasets(lama_df, spark_df)

Difference in col scaler__fillnamed__fillinf__AMT_ANNUITY and stats mean between -0.007084001321345568 (lama) and -0.006559545174241066 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_ANNUITY and stats min between -1.6944094896316528 (lama) and -1.6801564693450928 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_ANNUITY and stats 25% between -0.7266814112663269 (lama) and -0.7203020453453064 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_ANNUITY and stats 50% between -0.16115762293338776 (lama) and -0.15937957167625427 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_CREDIT and stats mean between -0.008321045897901058 (lama) and -0.0036277035251259804 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_CREDIT and stats min between -1.3564391136169434 (lama) and -1.3421242237091064 (spark)
Difference in col scaler__fillnamed__fillinf__AMT_CREDIT and stats 25% between -0.8055171966552734 (lama) and -0.7951341867446899 (spark)
Difference in col

KeyError: 'scaler__fillnamed__fillinf__logodds__oof__inter__(CODE_GENDER__NAME_CONTRACT_TYPE)'

# Nasa dataset

In [186]:
lama_df, spark_df = load_datasets('datalogs_nasa_dataset_42', 'datalog_lama_fit_predict_BoostLGBM.pickle', 'datalog_spark_fit_predict_BoostLGBM.pickle', True)

17
17
['sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_2', 'sensor_20', 'sensor_21', 'sensor_3', 'sensor_4', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9']
['sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_2', 'sensor_20', 'sensor_21', 'sensor_3', 'sensor_4', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9']

{'sensor_2', 'sensor_7', 'sensor_14', 'sensor_13', 'sensor_11', 'sensor_3', 'sensor_21', 'sensor_6', 'sensor_16', 'sensor_17', 'sensor_10', 'sensor_15', 'sensor_4', 'sensor_20', 'sensor_8', 'sensor_12', 'sensor_9'}

set()

set()


In [187]:
compare_datasets(lama_df, spark_df)

# Ailerons dataset

In [188]:
lama_df, spark_df = load_datasets('datalogs_ailerons_dataset_42', 'datalog_lama_fit_predict_BoostLGBM.pickle', 'datalog_spark_fit_predict_BoostLGBM.pickle', True)

27
29
['Se', 'SeTime1', 'SeTime10', 'SeTime11', 'SeTime12', 'SeTime2', 'SeTime4', 'SeTime6', 'SeTime8', 'Sgz', 'absRoll', 'alpha', 'climbRate', 'curPitch', 'curRoll', 'diffClb', 'diffDiffClb', 'diffRollRate', 'diffSeTime1', 'diffSeTime11', 'diffSeTime13', 'diffSeTime3', 'diffSeTime5', 'diffSeTime7', 'diffSeTime9', 'p', 'q']
['Se', 'SeTime1', 'SeTime10', 'SeTime11', 'SeTime12', 'SeTime13', 'SeTime2', 'SeTime4', 'SeTime6', 'SeTime7', 'SeTime8', 'Sgz', 'absRoll', 'alpha', 'climbRate', 'curPitch', 'curRoll', 'diffClb', 'diffDiffClb', 'diffRollRate', 'diffSeTime1', 'diffSeTime11', 'diffSeTime13', 'diffSeTime3', 'diffSeTime5', 'diffSeTime7', 'diffSeTime9', 'p', 'q']

{'SeTime2', 'Se', 'SeTime4', 'diffSeTime13', 'SeTime6', 'SeTime10', 'Sgz', 'curPitch', 'diffSeTime1', 'diffSeTime9', 'SeTime1', 'curRoll', 'p', 'SeTime11', 'SeTime8', 'diffSeTime5', 'climbRate', 'alpha', 'diffDiffClb', 'diffClb', 'SeTime12', 'diffSeTime7', 'absRoll', 'q', 'diffSeTime11', 'diffSeTime3', 'diffRollRate'}

set()

{'

In [189]:
compare_datasets(lama_df, spark_df)

Difference in col Sgz and stats mean between -12.599563598632812 (lama) and -12.599563598632812 (spark)
Difference in col Sgz and stats min between -99.0 (lama) and -99.0 (spark)
Difference in col Sgz and stats 25% between -29.0 (lama) and -29.0 (spark)
Difference in col Sgz and stats 50% between -13.0 (lama) and -13.0 (spark)
Difference in col absRoll and stats mean between -11.040218353271484 (lama) and -11.040218353271484 (spark)
Difference in col absRoll and stats min between -23.0 (lama) and -23.0 (spark)
Difference in col absRoll and stats 25% between -14.0 (lama) and -14.0 (spark)
Difference in col absRoll and stats 50% between -11.0 (lama) and -11.0 (spark)
Difference in col absRoll and stats 75% between -7.0 (lama) and -7.0 (spark)
Difference in col absRoll and stats max between -3.0 (lama) and -3.0 (spark)
Difference in col climbRate and stats mean between -11.102254867553711 (lama) and -11.102254867553711 (spark)
Difference in col climbRate and stats min between -975.0 (lama

# Phishing websites dataset

In [190]:
lama_df, spark_df = load_datasets('datalogs_phishing_websites_dataset_42', 'datalog_lama_fit_predict_BoostLGBM.pickle', 'datalog_spark_fit_predict_BoostLGBM.pickle', True)

30
30
['Abnormal_URL', 'DNSRecord', 'Domain_registeration_length', 'Favicon', 'Google_Index', 'HTTPS_token', 'Iframe', 'Links_in_tags', 'Links_pointing_to_page', 'Page_Rank', 'Prefix_Suffix', 'Redirect', 'Request_URL', 'RightClick', 'SFH', 'SSLfinal_State', 'Shortining_Service', 'Statistical_report', 'Submitting_to_email', 'URL_Length', 'URL_of_Anchor', 'age_of_domain', 'double_slash_redirecting', 'having_At_Symbol', 'having_IP_Address', 'having_Sub_Domain', 'on_mouseover', 'popUpWidnow', 'port', 'web_traffic']
['Abnormal_URL', 'DNSRecord', 'Domain_registeration_length', 'Favicon', 'Google_Index', 'HTTPS_token', 'Iframe', 'Links_in_tags', 'Links_pointing_to_page', 'Page_Rank', 'Prefix_Suffix', 'Redirect', 'Request_URL', 'RightClick', 'SFH', 'SSLfinal_State', 'Shortining_Service', 'Statistical_report', 'Submitting_to_email', 'URL_Length', 'URL_of_Anchor', 'age_of_domain', 'double_slash_redirecting', 'having_At_Symbol', 'having_IP_Address', 'having_Sub_Domain', 'on_mouseover', 'popUpWidn

In [191]:
compare_datasets(lama_df, spark_df)

Difference in col Abnormal_URL and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col DNSRecord and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col DNSRecord and stats 25% between -1.0 (lama) and -1.0 (spark)
Difference in col Domain_registeration_length and stats mean between -0.3367706835269928 (lama) and -0.3367706835269928 (spark)
Difference in col Domain_registeration_length and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col Domain_registeration_length and stats 25% between -1.0 (lama) and -1.0 (spark)
Difference in col Domain_registeration_length and stats 50% between -1.0 (lama) and -1.0 (spark)
Difference in col Favicon and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col Google_Index and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col HTTPS_token and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col Iframe and stats min between -1.0 (lama) and -1.0 (spark)
Difference in col Links_i

# kdd internet usage

In [192]:
lama_df, spark_df = load_datasets('datalogs_kdd_internet_usage_42', 'datalog_lama_fit_predict_BoostLGBM.pickle', 'datalog_spark_fit_predict_BoostLGBM.pickle', True)

77
77
['oof__le__Actual_Time', 'oof__le__Age', 'oof__le__Country', 'oof__le__Primary_Computing_Platform', 'oof__le__Primary_Language', 'oof__inter__(Country__Primary_Language)', 'oof__inter__(Country__Age)', 'oof__inter__(Country__Actual_Time)', 'oof__inter__(Primary_Language__Age)', 'oof__inter__(Primary_Language__Actual_Time)', 'oof__inter__(Age__Actual_Time)', 'oof__inter__(Country__Primary_Language__Age)', 'oof__inter__(Country__Primary_Language__Actual_Time)', 'oof__inter__(Country__Age__Actual_Time)', 'oof__inter__(Primary_Language__Age__Actual_Time)', 'Community_Membership_Family', 'Community_Membership_Hobbies', 'Community_Membership_None', 'Community_Membership_Other', 'Community_Membership_Political', 'Community_Membership_Professional', 'Community_Membership_Religious', 'Community_Membership_Support', 'Disability_Cognitive', 'Disability_Hearing', 'Disability_Motor', 'Disability_Not_Impaired', 'Disability_Vision', 'How_You_Heard_About_Survey_Banner', 'How_You_Heard_About_Surv

In [193]:
compare_datasets(lama_df, spark_df)

Difference in col oof__le__Actual_Time and stats mean between 0.26672986149787903 (lama) and 0.7300207018852234 (spark)
Difference in col oof__le__Actual_Time and stats min between 0.005124192219227552 (lama) and 0.2728385925292969 (spark)
Difference in col oof__le__Actual_Time and stats 25% between 0.07794380933046341 (lama) and 0.6560062766075134 (spark)
Difference in col oof__le__Actual_Time and stats 50% between 0.3365385830402374 (lama) and 0.6617757081985474 (spark)
Difference in col oof__le__Actual_Time and stats 75% between 0.34063470363616943 (lama) and 0.908187747001648 (spark)
Difference in col oof__le__Actual_Time and stats max between 0.7477937340736389 (lama) and 0.9951280951499939 (spark)
Difference in col oof__le__Age and stats mean between 0.26900652050971985 (lama) and 0.7274379134178162 (spark)
Difference in col oof__le__Age and stats min between 0.012441062368452549 (lama) and 0.5265557765960693 (spark)
Difference in col oof__le__Age and stats 25% between 0.17534694

KeyError: 'Disability_Motor'